In [ ]:
from modules.data_loading import load_train_data, load_listings_data
from modules.data_preprocessing import clean_data, feature_engineering
from modules.figure_generation import TSNE_kMeans_figure, random_forest_features
from modules.modeling import main, predict_price
import pandas as pd
import numpy as np
import ydata_profiling
import matplotlib.pyplot as plt
from joblib import load

# Loading and EDA

In [ ]:
df=load_train_data()
df_cleaned = clean_data(df)
df_engineered = feature_engineering(df_cleaned)
df_engineered.shape[0]

In [ ]:
# Extract quantitative columns
quantitatives = df_engineered.select_dtypes(include=['int64', 'float64']).copy()
# Drop the 'ID' column
quantitatives = quantitatives.drop(columns=['ID','price', 'rooms', 'bedrooms', 'washrooms', 'year_built',
                                            'has_certificate', 'year_certificate', 'due_certificate',
                                            'Plinthes électriques', 'Convecteurs',
                                            'Eau chaude', 'Air soufflé (pulsé)', 'Radiant', 'Thermopompe',
                                            'Gaz naturel', 'Poêle à bois', 'Foyer au gaz',
                                            'Porte de garage électrique',
                                            'Buanderie', 'Climatiseur', 'Aspirateur centrale', 'Spa',
                                            "Détecteur d'incendie (relié)", "Détecteur d'incendie (non relié)",
                                            'Adapté pour personne à mobilité réduite', 'Interphone', 'Fournaise',
                                            'Planchers chauffant', 'Ascenseur', "Échangeur d'air",
                                            "Système d'alarme", 'Borne de recharge', 'has_reno', 'last_year_reno', 'water_access',
                                            'min_price', 'max_price', 'fireplace_func', 'has_fireplace', 'has_pool', 'near_water', 'living_area'])


In [ ]:
TSNE_kMeans_figure(quantitatives)

In [ ]:
random_forest_features(quantitatives)

In [ ]:
quantitatives[quantitatives['cluster_label'] == 0].describe()

In [ ]:
quantitatives[quantitatives['cluster_label'] == 1].describe()

In [ ]:
quantitatives[quantitatives['cluster_label'] == 3].describe()

In [ ]:
quantitatives[quantitatives['cluster_label'] == 4].describe()

In [ ]:
quantitatives[quantitatives['cluster_label'] == 2].describe()

## Modelling:
### Predicting price

In [ ]:
# Using determinant features only
quantitatives['price'] = df_engineered['price']
df_feature_only = quantitatives.drop(columns=['land_eval', 'total_parking', 'cluster_label']).copy()
result_df = main(df_feature_only)

In [ ]:
result_df.head()

## EDA Profiling

In [ ]:
profile = df_feature_only.profile_report()
profile.to_file("eda_report.html")

## Predicting price for customer listings

In [ ]:
# Loading listings
listings = load_listings_data()
print(listings.shape[0])
list_cleaned = clean_data(listings)
list_engineered = feature_engineering(list_cleaned)
print(list_engineered.shape[0])

In [ ]:

# Price prediction
results = predict_price(list_engineered, model, scaler)